In [1]:
using Pkg
Pkg.activate("/Users/linussommer/signatures")

using AbstractAlgebra: derivative
using AlgebraicSolving
using AlgebraicSolving: _homogenize

# This is a first implementation of a signature based algorithm.
# It just substitutes the signature methods into the naive algorithm.
# Some homogenization is necessary.


"""
    Given a sigset return only the polynomial parts of the sigpairs.
"""
function natural(sigset)
    return [sigpair[2] for sigpair in sigset]
end;

"""
    Given a system with an additional variable x_{n+1} for homogenization
    evaluate all polynomials at (x1, x2, ..., xn, 1).
"""
function dehomogenize(system, subring, variables)
    # TODO deal with signatures
    systemnew = []
    for polynomial in system
        if parent(polynomial) != subring
            push!(systemnew, polynomial(variables..., 1))
        else
            push!(systemnew, polynomial)
        end
    end
    # This narrows the type form Vector{Any} to Vector{fpMPolyRingElem}
    systemnew = identity.(systemnew)
    return systemnew
end

"""
    Given a system with `n`` variables and `n` polynomial equations define a
    linear differential operator `patial`.
"""
function partial(polynomial, differentials)
    partial_polynomial = 0
    for (i, differential) in enumerate(differentials)
        partial_polynomial += differential * derivative(polynomial, i)
    end
    return partial_polynomial
end;

  Activating project at `~/signatures`


In [2]:
"""
    Old algorithm without signatures.
"""
function algorithm_i(q, ps)
    i = 0
    S = q
    g = q
    G = groebner_basis(Ideal(S))
    while true
        i += 1
        println("g = ", g)
        g = [partial(gi, ps) for gi in g]
        g = [normal_form(gi, Ideal(G)) for gi in g]
        if all(g .== 0)
            println(i, "o")
            return G
        else
            append!(S, g)
            G = groebner_basis(Ideal(S))
            println("G = ",G)
        end
    end    
end;

In [8]:
"""
    Given an ideal `I = <polynomials>` compute the Groebner basis of the 
    minimal ideal `J` s.t. `J` contains `I` and `J` is closed under `partial`.
"""
function naive_sig_algorithm(polynomials, differrentials, subring, variables)
    i = 0
    system = _homogenize(polynomials)
    g = polynomials # This is should also be homogenious maybe?
    groebnerbasis = sig_groebner_basis(system)
    while true
        i += 1
        g = [partial(gi, differrentials) for gi in g]
        # Without dehomogenizing the result does not correspond to the original algorithm
        nGB = dehomogenize(natural(groebnerbasis), subring, variables)
        # This does not use signatures
        g = [normal_form(gi, Ideal(nGB)) for gi in g]
        println("g = ", g)
        if all(g .== 0)
            println(i, "s")
            return groebnerbasis
        else
            append!(system, g) # push vs append?
            system = dehomogenize(system, subring, variables)
            system = _homogenize(system)
            # What happends to the previously computed signatures?
            groebnerbasis = sig_groebner_basis(system)
            println("G = ", natural(groebnerbasis))
        end
    end    
    return groebnerbasis
end;

In [9]:
R, (x1, x2, x3, x4, x5) = polynomial_ring(
    GF(65521), ["x$i" for i in 1:5], ordering=:degrevlex)
differrentials = [x3, x4, x5*x1, x5*x2 - 1, R(0)] 
polynomials = [x1^2 + x2^2 - 1];

In [10]:
G = naive_sig_algorithm(polynomials, differrentials, R, (x1, x2, x3, x4, x5))
G = natural(G)
G = dehomogenize(G, R, (x1, x2, x3, x4, x5))
G = groebner_basis(Ideal(G))
println(G)
println(length(G))

g = fpMPolyRingElem[2*x1*x3 + 2*x2*x4]
G = fpMPolyRingElem[x1^2 + x2^2 + 65520*x6^2, x1*x3 + x2*x4, x2^2*x3 + 65520*x1*x2*x4 + 65520*x3*x6^2]
g = fpMPolyRingElem[2*x3^2 + 2*x4^2 + 65519*x2 + 2*x5]
G = fpMPolyRingElem[x1^2 + x2^2 + 65520*x6^2, x1*x3 + x2*x4, x3^2 + x4^2 + 65520*x2*x6 + x5*x6, x2^2*x3 + 65520*x1*x2*x4 + 65520*x3*x6^2, x2*x3*x4 + 65520*x1*x4^2 + x1*x2*x6 + 65520*x1*x5*x6, x2^3*x6 + 65520*x2^2*x5*x6 + x4^2*x6^2 + 65520*x2*x6^3 + x5*x6^3, x1*x2^2*x6 + 65520*x1*x2*x5*x6 + x3*x4*x6^2]
g = fpMPolyRingElem[65515*x4]
G = fpMPolyRingElem[x4, x1^2 + x2^2 + 65520*x6^2, x1*x3 + x2*x4, x3^2 + x4^2 + 65520*x2*x6 + x5*x6, x2^2*x3 + 65520*x1*x2*x4 + 65520*x3*x6^2, x2*x3*x4 + 65520*x1*x4^2 + x1*x2*x6 + 65520*x1*x5*x6, x1*x2*x6 + 65520*x1*x5*x6, x2^3*x6 + 65520*x2^2*x5*x6 + x4^2*x6^2 + 65520*x2*x6^3 + x5*x6^3, x1*x2^2*x6 + 65520*x1*x2*x5*x6 + x3*x4*x6^2]
g = fpMPolyRingElem[65515*x2*x5 + 6]
G = fpMPolyRingElem[x4, x1^2 + x2^2 + 65520*x6^2, x1*x3 + x2*x4, x3^2 + x4^2 + 65520*x2*x6 + x5*x6,

In [6]:
# What I want:
# Also calling this breaks the above code somehow???
G1 = algorithm_i(polynomials, differrentials)
println(G1)
println(length(G1))

g = fpMPolyRingElem[x1^2 + x2^2 + 65520]
G = fpMPolyRingElem[x1*x3 + x2*x4, x1^2 + x2^2 + 65520, x2^2*x3 + 65520*x1*x2*x4 + 65520*x3]
g = fpMPolyRingElem[2*x1*x3 + 2*x2*x4]
G = fpMPolyRingElem[x3^2 + x4^2 + 65520*x2 + x5, x1*x3 + x2*x4, x1^2 + x2^2 + 65520, x2*x3*x4 + 65520*x1*x4^2 + x1*x2 + 65520*x1*x5, x2^2*x3 + 65520*x1*x2*x4 + 65520*x3, x2^3 + 65520*x2^2*x5 + x4^2 + 65520*x2 + x5, x1*x2^2 + 65520*x1*x2*x5 + x3*x4]
g = fpMPolyRingElem[2*x3^2 + 2*x4^2 + 65519*x2 + 2*x5]
G = fpMPolyRingElem[x4, x3^2 + 65520*x2 + x5, x1*x3, x1*x2 + 65520*x1*x5, x1^2 + x2^2 + 65520, x2^2*x3 + 65520*x3, x2^3 + 65520*x2^2*x5 + 65520*x2 + x5]
g = fpMPolyRingElem[65515*x4]
G = fpMPolyRingElem[x4, x2*x5 + 65520, x3^2 + 65520*x2 + x5, x2*x3 + 65520*x3*x5, x1*x3, x2^2 + x5^2 + 65519, x1*x2 + 65520*x1*x5, x1^2 + 65520*x5^2 + 1, x5^3 + x2 + 65519*x5, x3*x5^2 + 65520*x3, x1*x5^2 + 65520*x1]
g = fpMPolyRingElem[65515*x2*x5 + 6]
5o
fpMPolyRingElem[x4, x2*x5 + 65520, x3^2 + 65520*x2 + x5, x2*x3 + 65520*x3*x5, x1*x3,